PART A

In [68]:
import pandas as pd
from scipy.optimize import minimize

# Load the dataset from a CSV file
df = pd.read_csv(r'C:\Users\rahul\Desktop\MBAN\Models\price_response.csv')

# Filter the DataFrame to get the coefficients for Line 1 Product 1 and Line 1 Product 2
line_1_product_1_row = df[(df['Product'] == 'Line 1 Product 1')]
line_1_product_2_row = df[(df['Product'] == 'Line 1 Product 2')]
a1 = line_1_product_1_row['Intercept'].values[0]
b1 = line_1_product_2_row['Intercept'].values[0] 

# Define the objective function without regularization
def objective_function(prices, a1, b1):
    p1, p2 = prices
    return p1 * (a1 - b1 * p1) + p2 * (a1 - b1 * p2)

# Define the constraint function
def constraint_function(prices):
    p1, p2 = prices
    # Constraint: p2 - p1 >= 0
    return p2 - p1

# Set initial guess for prices (non-negative)
initial_prices = [169, 269]  # Adjust based on your scenario

# Define the constraint
constraint = {'type': 'ineq', 'fun': constraint_function}

# Define options with increased maximum number of iterations
options = {'maxiter': 1000}

# Minimize the objective function with bounds and constraints
result = minimize(objective_function, initial_prices, args=(a1, b1), method='COBYLA', constraints=constraint, options=options)

# Extract optimal prices
optimal_prices = result.x

print("Optimal prices (p1, p2):", optimal_prices)


35234.54578551236 37790.24083213697
Optimal prices (p1, p2): [ 700.90702691 1093.19810057]


In [35]:
from scipy.optimize import minimize

# Define the objective function without regularization
def objective_function(prices):
    p1, p2 = prices
    return p1 * (a1 - b1 * p1) + p2 * (a1 - b1 * p2)

# Define the constraint function
def constraint_function(prices):
    p1, p2 = prices
    # Constraint: p2 - p1 >= 0
    return p2 - p1

# Load the dataset
dataset = {
    "Line 1 Product 1": (35234.54579, -45.89644971, 80020),
    "Line 1 Product 2": (37790.24083, -8.227794173, 89666),
    "Line 1 Product 3": (35675.33322, -7.58443641, 80638),
    "Line 2 Product 1": (37041.38038, -9.033166404, 86740),
    "Line 2 Product 2": (36846.14039, -4.427869206, 84050),
    "Line 2 Product 3": (35827.02375, -2.629060015, 86565),
    "Line 3 Product 1": (39414.26632, -2.421483918, 87051),
    "Line 3 Product 2": (35991.95146, -4.000512401, 85156),
    "Line 3 Product 3": (39313.31703, -2.296622373, 87588),
}

# Extract coefficients for the EvoTech Pro Series
a1 = dataset["Line 1 Product 1"][0]
b1 = dataset["Line 1 Product 2"][0]

# Set initial guess for prices (non-negative)
initial_prices = [169, 269]

# Define bounds for prices (non-negative)
#bounds = [(0, None), (0, None)]

# Define the constraint
constraint = {'type': 'ineq', 'fun': constraint_function}

# Define options with increased maximum number of iterations
options = {'maxiter': 1000}  # Increase the maximum number of iterations

# Minimize the objective function (maximize the objective) with bounds and constraints
result = minimize(objective_function, initial_prices, method='COBYLA', constraints=constraint, options=options)

# Extract optimal prices
optimal_prices = result.x

print("Optimal prices (p1, p2):", optimal_prices)


Optimal prices (p1, p2): [ 706.89649733 1088.71836531]


Part B

In [54]:
from gurobipy import Model, GRB

# Define the objective function
def objective_function(prices, a1, b1):
    p1, p2 = prices
    return -(p1 * (a1 - b1 * p1) + p2 * (a1 - b1 * p2))

# Define the constraint function
def constraint_function(prices):
    p1, p2 = prices
    return p2 - p1

import numpy as np

# Define the projection step using Gurobi quadratic programming
def projection_step(prices):
    # Initialize Gurobi model
    m = Model("projection_step")
    
    # Define variables
    p1 = m.addVar(lb=0, name="p1")
    p2 = m.addVar(lb=0, name="p2")
    
    # Define objective function (minimize 0)
    m.setObjective(0, GRB.MINIMIZE)
    
    # Define constraint: p2 - p1 >= 0
    m.addConstr(p2 - p1, GRB.GREATER_EQUAL, 0)

    # Set initial values for variables
    p1.start = prices[0]
    p2.start = prices[1]
    
    # Solve the model
    m.optimize()
    
    # Extract optimal prices
    optimal_prices = [p1.x, p2.x]
    
    return optimal_prices

# Set initial guess for prices
initial_prices = np.zeros(2)

# Set step size for gradient descent
step_size = 0.001

# Set stopping criterion
stopping_criterion = 1e-6

# Coefficients for the EvoTech Pro Series
a1 = 35234.54579
b1 = -45.89644971

# Perform projected gradient descent
while True:
    # Compute gradient of the objective function
    gradient = np.array([-a1 + 2 * b1 * initial_prices[0], -a1 + 2 * b1 * initial_prices[1]])
    
    # Update prices using gradient descent step
    next_prices = initial_prices - step_size * gradient
    
    # Project prices to satisfy the constraint
    next_prices = projection_step(next_prices)
    
    # Check stopping criterion
    if np.linalg.norm(next_prices - initial_prices) < stopping_criterion:
        break
    
    initial_prices = next_prices

print("Optimal prices (p1, p2):", next_prices)


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: AMD Ryzen 7 5800U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 1 rows, 2 columns and 2 nonzeros
Model fingerprint: 0xae00555b
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Presolve removed 1 rows and 2 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  0.000000000e+00
Optimal prices (p1, p2): [0.0, 0.0]


C:\Users\rahul\AppData\Local\Temp\ipykernel_30524\4244913456.py:30: DeprecationWarning: Deprecated, pass a TempConstr or use Model.addLConstr
  m.addConstr(p2 - p1, GRB.GREATER_EQUAL, 0)


PART C

In [55]:
from gurobipy import Model, GRB

# Coefficients for each product line and version
coefficients = {
    "ProductLine1": {
        "Basic": {"a": 1000, "b": 0.1},
        "Advanced": {"a": 1500, "b": 0.15},
        "Premium": {"a": 2000, "b": 0.2}
    },
    "ProductLine2": {
        "Basic": {"a": 1200, "b": 0.12},
        "Advanced": {"a": 1700, "b": 0.17},
        "Premium": {"a": 2200, "b": 0.22}
    },
    "ProductLine3": {
        "Basic": {"a": 1100, "b": 0.11},
        "Advanced": {"a": 1600, "b": 0.16},
        "Premium": {"a": 2100, "b": 0.21}
    }
}

# Initialize Gurobi model
model = Model("QuadraticOptimization")

# Decision variables
prices = {}
for product_line, versions in coefficients.items():
    for version, coeffs in versions.items():
        prices[product_line, version] = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name=f"price_{product_line}_{version}")

# Auxiliary binary variables to enforce increasing order constraint
is_increasing = {}
for product_line in coefficients.keys():
    is_increasing[product_line] = model.addVars(["Basic_Advanced", "Advanced_Premium"], vtype=GRB.BINARY)

# Objective function: maximize revenue
model.setObjective(sum(prices[product_line, version] * (coeffs["a"] - coeffs["b"] * prices[product_line, version])
                      for product_line, versions in coefficients.items() for version, coeffs in versions.items()),
                   sense=GRB.MAXIMIZE)

# Constraints: Prices of Basic, Advanced, and Premium versions within each product line must be increasing
for product_line in coefficients.keys():
    model.addConstr(prices[product_line, "Basic"] <= prices[product_line, "Advanced"])
    model.addConstr(prices[product_line, "Advanced"] <= prices[product_line, "Premium"])
    
    # Auxiliary constraints to enforce increasing order
    model.addConstr(prices[product_line, "Basic"] + (1 - is_increasing[product_line]["Basic_Advanced"]) * 10000 >= prices[product_line, "Advanced"])
    model.addConstr(prices[product_line, "Advanced"] + (1 - is_increasing[product_line]["Advanced_Premium"]) * 10000 >= prices[product_line, "Premium"])

# Optimize the model
model.optimize()

# Display optimal prices
print("Optimal Prices:")
for product_line, versions in coefficients.items():
    for version in versions.keys():
        print(f"{product_line} - {version}: {prices[product_line, version].X}")

# Display optimal revenue
print(f"\nOptimal Revenue: {model.objVal}")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: AMD Ryzen 7 5800U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 12 rows, 15 columns and 30 nonzeros
Model fingerprint: 0xc159dfbd
Model has 9 quadratic objective terms
Variable types: 9 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+03, 2e+03]
  QObjective range [2e-01, 4e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+04, 1e+04]
Found heuristic solution: objective -0.0000000
Presolve removed 8 rows and 12 columns
Presolve time: 0.00s
Presolved: 4 rows, 3 columns, 8 nonzeros
Presolved model has 3 quadratic objective terms
Found heuristic solution: objective 2.400000e+07
Variable types: 3 continuous, 0 integer (0 binary)
Root relaxation presolve removed 2 rows and 0 columns
Root relaxation presolve time: 0.0

PART D

In [96]:
from gurobipy import Model, GRB
import pandas as pd

# Load the dataset from a CSV file
df = pd.read_csv(r'C:\Users\rahul\Desktop\MBAN\Models\price_response.csv')

# Create a Gurobi model
model = Model("Price Optimization")

# Decision variables: price for each product in each line
prices = {}
for product, intercept, sensitivity, _ in df.itertuples(index=False):
    prices[product] = model.addVar(lb=0.0, name=f"price_{product}")

# Objective: maximize total revenue
total_revenue = 0
for product, intercept, sensitivity, _ in df.itertuples(index=False):
    total_revenue += prices[product] * (intercept + sensitivity * prices[product])
model.setObjective(total_revenue, GRB.MAXIMIZE)

# Constraints: Prices within each line must be increasing
for product, group in df.groupby('Product'):
    products_in_line = group['Product'].tolist()
    for i in range(len(products_in_line)-1):
        model.addConstr(prices[products_in_line[i+1]] >= prices[products_in_line[i]])

# Optimize the model
model.optimize()

# Display results
if model.status == GRB.OPTIMAL:
    print("***********PART D***********")
    print(f"Optimal objective {model.objVal}")
    for var in model.getVars():
        print(f"{var.varName}: {var.x}")
    print("Optimal Revenue:", model.objVal)
else:
    
    print("***********PART D***********")
    print("No solution found.")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: AMD Ryzen 7 5800U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 0 rows, 9 columns and 0 nonzeros
Model fingerprint: 0xfa548505
Model has 9 quadratic objective terms
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [4e+04, 4e+04]
  QObjective range [5e+00, 9e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Presolve removed 0 rows and 9 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Barrier solved model in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective 7.38368672e+08
***********PART D***********
Optimal objective 738368671.9632478
price_Line 1 Product 1: 383.84827160837136
price_Line 1 Product 2: 2296.498918132054
price_Line 1 Product 3: 2351.877667019544
price_Line 2 Product 1: 2050.2987944234346

D Hardcoded

In [98]:
from gurobipy import Model, GRB

# Data: Product, Intercept, Sensitivity, Capacity
data = {
    ("Line 1", "Product 1"): (35234.54579, -45.89644971, 80020),
    ("Line 1", "Product 2"): (37790.24083, -8.227794173, 89666),
    ("Line 1", "Product 3"): (35675.33322, -7.58443641, 80638),
    ("Line 2", "Product 1"): (37041.38038, -9.033166404, 86740),
    ("Line 2", "Product 2"): (36846.14039, -4.427869206, 84050),
    ("Line 2", "Product 3"): (35827.02375, -2.629060015, 86565),
    ("Line 3", "Product 1"): (39414.26632, -2.421483918, 87051),
    ("Line 3", "Product 2"): (35991.95146, -4.000512401, 85156),
    ("Line 3", "Product 3"): (39313.31703, -2.296622373, 87588),
}

# Create a Gurobi model
model = Model("Price Optimization")

# Decision variables: price for each product in each line
prices = {}
for line, product in data.keys():
    prices[line, product] = model.addVar(lb=0.0, name=f"price_{line}_{product}")

# Objective: maximize total revenue
total_revenue = 0
for line, product in data.keys():
    intercept, sensitivity, capacity = data[(line, product)]
    total_revenue += prices[line, product] * (intercept + sensitivity * prices[line, product])
model.setObjective(total_revenue, GRB.MAXIMIZE)

# Constraints: Prices within each line must be increasing
for line in set(line for line, _ in data.keys()):
    products_in_line = [product for l, product in data.keys() if l == line]
    for i in range(len(products_in_line)-1):
        model.addConstr(prices[line, products_in_line[i+1]] >= prices[line, products_in_line[i]])

# Optimize the model
model.optimize()

# Display results
if model.status == GRB.OPTIMAL:
    for var in model.getVars():
        print(f"{var.varName}: {var.x}")
    print("Optimal Revenue:", model.objVal)
else:
    print("No solution found.")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: AMD Ryzen 7 5800U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 6 rows, 9 columns and 12 nonzeros
Model fingerprint: 0x3a1285f4
Model has 9 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+04, 4e+04]
  QObjective range [5e+00, 9e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Presolve time: 0.00s
Presolved: 6 rows, 9 columns, 12 nonzeros
Presolved model has 9 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 3.000e+00
 Factor NZ  : 9.000e+00
 Factor Ops : 1.500e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   5.34101234e+08  1.18618490e

ALL DYNAMIC

In [97]:
import pandas as pd
from scipy.optimize import minimize
from gurobipy import Model, GRB
import numpy as np

# Load the dataset from a CSV file
df = pd.read_csv(r'C:\Users\rahul\Desktop\MBAN\Models\price_response.csv')

# Filter the DataFrame to get the coefficients for Line 1 Product 1 and Line 1 Product 2
line_1_product_1_row = df[(df['Product'] == 'Line 1 Product 1')]
line_1_product_2_row = df[(df['Product'] == 'Line 1 Product 2')]
a1 = line_1_product_1_row['Intercept'].values[0]
b1 = line_1_product_2_row['Intercept'].values[0] 

# Define the objective function without regularization
def objective_function(prices, a1, b1):
    p1, p2 = prices
    return p1 * (a1 - b1 * p1) + p2 * (a1 - b1 * p2)

# Define the constraint function
def constraint_function(prices):
    p1, p2 = prices
    # Constraint: p2 - p1 >= 0
    return p2 - p1

# Define the projection step using Gurobi quadratic programming
def projection_step(prices):
    # Initialize Gurobi model
    m = Model("projection_step")
    
    # Define variables
    p1 = m.addVar(lb=0, name="p1")
    p2 = m.addVar(lb=0, name="p2")
    
    # Define objective function (minimize 0)
    m.setObjective(0, GRB.MINIMIZE)
    
    # Define constraint: p2 - p1 >= 0
    m.addConstr(p2 - p1, GRB.GREATER_EQUAL, 0)

    # Set initial values for variables
    p1.start = prices[0]
    p2.start = prices[1]
    
    # Solve the model
    m.optimize()
    
    # Extract optimal prices
    optimal_prices = [p1.x, p2.x]
    
    return optimal_prices


# Set initial guess for prices (non-negative)
initial_prices = [169, 269]  # Adjust based on your scenario

# Define the constraint
constraint = {'type': 'ineq', 'fun': constraint_function}

# Define options with increased maximum number of iterations
options = {'maxiter': 1000}

# Minimize the objective function with bounds and constraints
result = minimize(objective_function, initial_prices, args=(a1, b1), method='COBYLA', constraints=constraint, options=options)

# Extract optimal prices
optimal_prices = result.x

print("*********PART A**********")
print("Optimal prices (p1, p2):", optimal_prices)

# Set initial guess for prices
b_initial_prices = np.zeros(2)

# Set step size for gradient descent
step_size = 0.001

# Set stopping criterion
stopping_criterion = 1e-6

# Perform projected gradient descent
while True:
    
    # Compute gradient of the objective function
    gradient = np.array([-a1 + 2 * b1 * b_initial_prices[0], -a1 + 2 * b1 * b_initial_prices[1]])
    
    # Update prices using gradient descent step
    next_prices = b_initial_prices - step_size * gradient
    
    # Project prices to satisfy the constraint
    next_prices = projection_step(next_prices)
    
    # Check stopping criterion
    if np.linalg.norm(next_prices - b_initial_prices) < stopping_criterion:
        break
    
    b_initial_prices = next_prices
print("*********PART B**********")

print("Optimal prices (p1, p2):", next_prices)

#*************************************PART C*************************************
# Coefficients for each product line and version
coefficients = {
    "ProductLine1": {
        "Basic": {"a": 1000, "b": 0.1},
        "Advanced": {"a": 1500, "b": 0.15},
        "Premium": {"a": 2000, "b": 0.2}
    },
    "ProductLine2": {
        "Basic": {"a": 1200, "b": 0.12},
        "Advanced": {"a": 1700, "b": 0.17},
        "Premium": {"a": 2200, "b": 0.22}
    },
    "ProductLine3": {
        "Basic": {"a": 1100, "b": 0.11},
        "Advanced": {"a": 1600, "b": 0.16},
        "Premium": {"a": 2100, "b": 0.21}
    }
}
# Initialize Gurobi model
model = Model("QuadraticOptimization")

# Decision variables
prices = {}
for product_line, versions in coefficients.items():
    for version, coeffs in versions.items():
        prices[product_line, version] = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name=f"price_{product_line}_{version}")

# Auxiliary binary variables to enforce increasing order constraint
is_increasing = {}
for product_line in coefficients.keys():
    is_increasing[product_line] = model.addVars(["Basic_Advanced", "Advanced_Premium"], vtype=GRB.BINARY)

# Objective function: maximize revenue
model.setObjective(sum(prices[product_line, version] * (coeffs["a"] - coeffs["b"] * prices[product_line, version])
                      for product_line, versions in coefficients.items() for version, coeffs in versions.items()),
                   sense=GRB.MAXIMIZE)

# Constraints: Prices of Basic, Advanced, and Premium versions within each product line must be increasing
for product_line in coefficients.keys():
    model.addConstr(prices[product_line, "Basic"] <= prices[product_line, "Advanced"])
    model.addConstr(prices[product_line, "Advanced"] <= prices[product_line, "Premium"])
    
    # Auxiliary constraints to enforce increasing order
    model.addConstr(prices[product_line, "Basic"] + (1 - is_increasing[product_line]["Basic_Advanced"]) * 10000 >= prices[product_line, "Advanced"])
    model.addConstr(prices[product_line, "Advanced"] + (1 - is_increasing[product_line]["Advanced_Premium"]) * 10000 >= prices[product_line, "Premium"])

# Optimize the model
model.optimize()

# Display optimal prices
print("\n*********PART C**********")
print("Optimal Prices:")
for product_line, versions in coefficients.items():
    for version in versions.keys():
        print(f"{product_line} - {version}: {prices[product_line, version].X}")

# Display optimal revenue
print(f"\nOptimal Revenue: {model.objVal}")


#*************************************PART D*************************************

# Create a Gurobi model
model = Model("Price Optimization")

# Decision variables: price for each product in each line
prices = {}
for product, intercept, sensitivity, _ in df.itertuples(index=False):
    prices[product] = model.addVar(lb=0.0, name=f"price_{product}")

# Objective: maximize total revenue
total_revenue = 0
for product, intercept, sensitivity, _ in df.itertuples(index=False):
    total_revenue += prices[product] * (intercept + sensitivity * prices[product])
model.setObjective(total_revenue, GRB.MAXIMIZE)

# Constraints: Prices within each line must be increasing
for product, group in df.groupby('Product'):
    products_in_line = group['Product'].tolist()
    for i in range(len(products_in_line)-1):
        model.addConstr(prices[products_in_line[i+1]] >= prices[products_in_line[i]])

# Optimize the model
model.optimize()

# Display results
if model.status == GRB.OPTIMAL:
    print("***********PART D***********")
    print(f"Optimal objective {model.objVal}")
    for var in model.getVars():
        print(f"{var.varName}: {var.x}")
    print("Optimal Revenue:", model.objVal)
else:
    
    print("***********PART D***********")
    print("No solution found.")


*********PART A**********
Optimal prices (p1, p2): [ 700.90702691 1093.19810057]
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: AMD Ryzen 7 5800U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 1 rows, 2 columns and 2 nonzeros
Model fingerprint: 0xacd2cf7b
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Presolve removed 1 rows and 2 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  0.000000000e+00


*********PART B**********
Optimal prices (p1, p2): [0.0, 0.0]
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: AMD Ryzen 7 5800U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 12 rows, 15 columns and 30 nonzeros
Model fingerprint: 0xc159dfbd
Model has 9 quadratic objective terms
Variable types: 9 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+03, 2e+03]
  QObjective range [2e-01, 4e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+04, 1e+04]
Found heuristic solution: objective -0.0000000
Presolve removed 8 rows and 12 columns
Presolve time: 0.00s
Presolved: 4 rows, 3 columns, 8 nonzeros
Presolved model has 3 quadratic objective terms
Found heuristic solution: objective 2.400000e+07
Variable types: 3 continuous, 0 integer (0 binary)
Root relaxation presolve r

C:\Users\rahul\AppData\Local\Temp\ipykernel_30524\3026201046.py:39: DeprecationWarning: Deprecated, pass a TempConstr or use Model.addLConstr
  m.addConstr(p2 - p1, GRB.GREATER_EQUAL, 0)


ALL HARDCODED